In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Cross-validated CCA (ViT PCs ↔ brain PCs) with bootstrap CIs, permutation null, and FDR.

Pipeline per area:
  1) Repeat-split (even/odd) to remove trial noise.
  2) Outer K-fold over stimuli (default 5).
  3) Fit brain PCA on TRAIN stimuli (even repeats), project ALL images (even/odd).
  4) Fit CCA on TRAIN stimuli: (Zv_train, Zb_train_even).
  5) Evaluate canonical correlations on TEST stimuli using ODD repeats:
        (Xc_test, Yc_test) = cca.transform(Zv_test, Zb_test_odd)
        r_test[i] = corr(Xc_test[:, i], Yc_test[:, i]).
  6) Bootstrap CIs over TEST stimuli (no refit).
  7) Permutation null: shuffle TRAIN pairing between Zv and Zb, refit CCA, evaluate on the same TEST split.
  8) Aggregate across folds: mean spectrum, pool null across folds, p-values, FDR.
  9) Save figures and .npz artifacts.

Author: Maria + Pläku 🐾
"""

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cross_decomposition import CCA
from scipy.special import softmax
from skbio.stats.composition import clr

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREAS       = ["VISp", "VISl", "VISrl", "VISal", "VISam", "VISpm"]
N_IMAGES, N_TRIALS = 118, 50
VAR_VIT, VAR_BRAIN = 0.90, 0.90

K_OUTER    = 5       # stimulus-level CV folds
N_BOOT     = 500     # bootstrap resamples over TEST stimuli
N_PERM     = 200     # permutation null (per fold; refits CCA)
N_CCA_MAX  = 20      # max canonical comps to report (also bounded by dims)
RANDOM_SEED = 42

OUTDIR     = "results_cvcca"
os.makedirs(OUTDIR, exist_ok=True)
rng = np.random.default_rng(RANDOM_SEED)

# -----------------------------
# Helpers
# -----------------------------
def vit_pcs(vit_logits: np.ndarray, var=0.90) -> Tuple[np.ndarray, PCA, int]:
    """CLR(+eps) → PCA → standardized PCs."""
    Xv = clr(softmax(np.asarray(vit_logits), axis=1) + 1e-12)
    pfull = PCA(random_state=RANDOM_SEED).fit(Xv)
    n = int(np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var) + 1)
    p = PCA(n_components=n, random_state=RANDOM_SEED).fit(Xv)
    Zv = p.transform(Xv)
    # stabilize
    Zv = (Zv - Zv.mean(0)) / (Zv.std(0) + 1e-8)
    return Zv, p, n

def repeat_split_area(dat_area: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """(neurons × images × trials × time) -> (even_mean, odd_mean) as (neurons × images)."""
    idx = np.arange(N_TRIALS)
    even, odd = idx[::2], idx[1::2]
    X_even = dat_area[:, :, even, :].mean(axis=(2,3))
    X_odd  = dat_area[:, :, odd,  :].mean(axis=(2,3))
    return X_even, X_odd

def brain_pca_on_train(X_even: np.ndarray, img_tr: np.ndarray, X_odd: np.ndarray, var=0.90) -> Tuple[np.ndarray, np.ndarray, PCA, int]:
    """
    Fit PCA on TRAIN images using EVEN repeats; project EVEN/ODD for ALL images into that basis.
    Return (Ze_all, Zo_all) of shape (N_IMAGES × ncomp).
    """
    pfull = PCA(random_state=RANDOM_SEED).fit(X_even[:, img_tr].T)
    n = int(np.searchsorted(np.cumsum(pfull.explained_variance_ratio_), var) + 1)
    p = PCA(n_components=n, random_state=RANDOM_SEED).fit(X_even[:, img_tr].T)

    Ze_all = (X_even.T - p.mean_) @ p.components_.T
    Zo_all = (X_odd.T  - p.mean_) @ p.components_.T

    Ze_all -= Ze_all.mean(0); Zo_all -= Zo_all.mean(0)
    return Ze_all, Zo_all, p, n

def canonical_corrs(Xc: np.ndarray, Yc: np.ndarray) -> np.ndarray:
    """Pearson correlations for paired canonical variables."""
    Xc = np.asarray(Xc); Yc = np.asarray(Yc)
    d = min(Xc.shape[1], Yc.shape[1])
    r = np.empty(d)
    for i in range(d):
        x = Xc[:, i] - Xc[:, i].mean()
        y = Yc[:, i] - Yc[:, i].mean()
        num = float(np.dot(x, y))
        den = float(np.sqrt(np.dot(x, x) * np.dot(y, y)) + 1e-12)
        r[i] = num / den
    return r

def fdr_bh(p: np.ndarray, q=0.05) -> Tuple[np.ndarray, float]:
    """Benjamini–Hochberg FDR (returns boolean mask and cutoff)."""
    p = np.asarray(p)
    m = len(p)
    order = np.argsort(p)
    thresh = q * (np.arange(1, m+1)) / m
    passed = p[order] <= thresh
    if not np.any(passed):
        return np.zeros_like(p, dtype=bool), 0.0
    k = np.max(np.where(passed)[0]) + 1
    cutoff = p[order[k-1]]
    return p <= cutoff, float(cutoff)

# -----------------------------
# Load data
# -----------------------------
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']
Zv, vit_pca, vit_n = vit_pcs(vit_logits, VAR_VIT)
print(f"ViT PCs covering {int(VAR_VIT*100)}% variance: {vit_n}")

dat   = np.load(NEURAL_PATH, mmap_mode='r')               # (neurons × (images*trials*time))
areas = np.load(AREAS_PATH, allow_pickle=True)

# -----------------------------
# Per-area CV-CCA
# -----------------------------
for AREA in AREAS:
    mask = (areas == AREA)
    dA = dat[mask]
    if dA.size == 0:
        print(f"[WARN] No data for {AREA}; skipping.")
        continue

    print(f"\n=== Area: {AREA} ===")
    n_neu, n_total = dA.shape
    n_time = n_total // (N_IMAGES * N_TRIALS)
    dA = dA.reshape(n_neu, N_IMAGES, N_TRIALS, n_time)

    # repeat split
    X_even, X_odd = repeat_split_area(dA)

    # outer K-fold over stimuli
    stim_idx = np.arange(N_IMAGES)
    kf = KFold(n_splits=K_OUTER, shuffle=True, random_state=RANDOM_SEED)

    # storage across folds
    fold_real_corrs: List[np.ndarray] = []
    fold_null_means: List[np.ndarray] = []
    pooled_null_corrs: List[np.ndarray] = []  # collect all null draws across folds

    for fold, (idx_tr, idx_te) in enumerate(kf.split(stim_idx), 1):
        img_tr = stim_idx[idx_tr]
        img_te = stim_idx[idx_te]

        # brain PCA in TRAIN (even repeats)
        Ze_all, Zo_all, brain_pca, brain_n = brain_pca_on_train(X_even, img_tr, X_odd, VAR_BRAIN)

        # choose CCA dimensionality
        ncca = min(N_CCA_MAX, vit_n, brain_n, len(img_tr), len(img_te))
        if ncca < 1:
            print(f"[Fold {fold}] ncca<1, skipping fold.")
            continue

        # --- Fit CCA on TRAIN stimuli ---
        cca = CCA(n_components=ncca, max_iter=1000)
        cca.fit(Zv[img_tr], Ze_all[img_tr])

        # --- Evaluate on TEST stimuli (use ODD repeats for independent noise) ---
        Xc_te, Yc_te = cca.transform(Zv[img_te], Zo_all[img_te])
        r_test = canonical_corrs(Xc_te, Yc_te)  # length ncca
        fold_real_corrs.append(r_test)

        # --- Bootstrap over TEST stimuli (no refit) ---
        boot_corrs = np.zeros((N_BOOT, ncca))
        for b in range(N_BOOT):
            idx = rng.integers(0, len(img_te), len(img_te))
            xb = Xc_te[idx]; yb = Yc_te[idx]
            boot_corrs[b] = canonical_corrs(xb, yb)

        # --- Permutation null: shuffle TRAIN pairing, refit CCA, eval on same TEST ---
        null_corrs = np.zeros((N_PERM, ncca))
        Zb_train = Ze_all[img_tr]
        for b in range(N_PERM):
            perm = rng.permutation(len(img_tr))
            cca_n = CCA(n_components=ncca, max_iter=1000)
            cca_n.fit(Zv[img_tr], Zb_train[perm])     # break stimulus alignment on TRAIN
            Xc_n, Yc_n = cca_n.transform(Zv[img_te], Zo_all[img_te])
            null_corrs[b] = canonical_corrs(Xc_n, Yc_n)

        # summary (per fold)
        null_mean = null_corrs.mean(0)
        fold_null_means.append(null_mean)
        pooled_null_corrs.append(null_corrs)  # pool for area-level p-values

        # save fold plot
        lo, hi = np.percentile(boot_corrs, [2.5, 97.5], axis=0)
        lo_n, hi_n = np.percentile(null_corrs, [2.5, 97.5], axis=0)
        x = np.arange(1, ncca+1)

        plt.figure(figsize=(7,5))
        plt.plot(x, r_test, 'o-', label='Test canonical corr (real)')
        plt.fill_between(x, lo, hi, alpha=0.20, label='Real 95% CI (bootstrap)')
        plt.plot(x, null_mean, 'o--', label='Null mean (perm)')
        plt.fill_between(x, lo_n, hi_n, alpha=0.20, color='gray', label='Null 95% band')
        plt.title(f"{AREA} — CCA fold {fold}  (ncca={ncca}, brainPCs={brain_n})")
        plt.xlabel("Canonical component")
        plt.ylabel("Correlation on held-out stimuli")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(OUTDIR, f"{AREA}_fold{fold}_cvcca.png"), dpi=150)
        plt.close()

        # save fold bundle
        np.savez(os.path.join(OUTDIR, f"{AREA}_fold{fold}_cvcca.npz"),
                 r_test=r_test, boot_corrs=boot_corrs, null_corrs=null_corrs,
                 img_tr=img_tr, img_te=img_te, ncca=ncca, brain_n=brain_n)

    # ---- Aggregate across folds ----
    if len(fold_real_corrs) == 0:
        print(f"[Area {AREA}] no valid folds.")
        continue

    # Make same length across folds
    L = min(len(r) for r in fold_real_corrs)
    real_mat = np.stack([r[:L] for r in fold_real_corrs], axis=0)  # (folds × L)
    real_mean = real_mat.mean(0)

    # Pool null across folds (concat along samples, then trim to L)
    null_all = np.concatenate([n[:, :L] for n in pooled_null_corrs], axis=0)  # (folds*N_PERM × L)
    null_mean = null_all.mean(0)

    # p-values per component: compare real_mean to distribution of null means.
    # Build null distribution of mean-by-fold by resampling null rows to groups of size N_PERM (approx):
    # Simpler: treat each null row as an independent draw of the "mean" (conservative).
    p_vals = np.array([(1 + np.sum(null_all[:, j] >= real_mean[j])) / (1 + null_all.shape[0]) for j in range(L)])

    # FDR across components
    sig_mask, cutoff = fdr_bh(p_vals, q=0.05)

    # Save across-fold plot
    x = np.arange(1, L+1)
    plt.figure(figsize=(7,5))
    for r in real_mat:
        plt.plot(x, r, color='lightsteelblue', alpha=0.5)
    plt.plot(x, real_mean, 'o-', color='tab:blue', label='Real mean (across folds)')
    plt.plot(x, null_mean, 'o--', color='tab:gray', label='Null mean (pooled)')
    # mark significant
    for j in range(L):
        if sig_mask[j]:
            plt.axvline(j+1, ymin=0, ymax=0.05, color='crimson', lw=2, alpha=0.7)
    plt.title(f"{AREA} — CV-CCA across folds (FDR<0.05 comps marked)")
    plt.xlabel("Canonical component")
    plt.ylabel("Correlation on held-out stimuli")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, f"{AREA}_cvcca_acrossfolds.png"), dpi=150)
    plt.close()

    # Print concise summary
    top = min(5, L)
    print(f"[Area {AREA}] top-{top} mean test canonical corr: {np.round(real_mean[:top], 3)}")
    print(f"[Area {AREA}] p-values (first {top}): {np.round(p_vals[:top], 4)}")
    print(f"[Area {AREA}] FDR<0.05 significant comps: {np.where(sig_mask)[0]+1}")

    # Save summary
    np.savez(os.path.join(OUTDIR, f"{AREA}_cvcca_summary.npz"),
             real_mean=real_mean, null_mean=null_mean,
             p_vals=p_vals, sig_mask=sig_mask, fdr_cutoff=cutoff)


ViT PCs covering 90% variance: 44

=== Area: VISp ===
[Area VISp] top-5 mean test canonical corr: [ 0.238  0.131  0.08  -0.027  0.268]
[Area VISp] p-values (first 5): [0.1758 0.2707 0.3516 0.5425 0.1089]
[Area VISp] FDR<0.05 significant comps: []

=== Area: VISl ===
[Area VISl] top-5 mean test canonical corr: [0.33  0.131 0.16  0.137 0.05 ]
[Area VISl] p-values (first 5): [0.0659 0.3047 0.2468 0.2428 0.4006]
[Area VISl] FDR<0.05 significant comps: []

=== Area: VISrl ===
[Area VISrl] top-5 mean test canonical corr: [ 0.116  0.039  0.058 -0.009  0.016]
[Area VISrl] p-values (first 5): [0.2617 0.4416 0.3976 0.5165 0.4785]
[Area VISrl] FDR<0.05 significant comps: []

=== Area: VISal ===
[Area VISal] top-5 mean test canonical corr: [0.267 0.233 0.031 0.011 0.063]
[Area VISal] p-values (first 5): [0.1179 0.1269 0.4366 0.5025 0.3826]
[Area VISal] FDR<0.05 significant comps: []

=== Area: VISam ===
[Area VISam] top-5 mean test canonical corr: [0.194 0.213 0.002 0.257 0.039]
[Area VISam] p-val